# Long Term Credit Rating Projection 

## Import of all the packages needed

In [1]:
import pandas as pd
import numpy as np
!pip install tensorflow
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.imputation.mice import MICE, MICEData
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.constraints import maxnorm
from matplotlib import pyplot
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
# import fancyimpute
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
# from imblearn.over_sampling import SMOTE
import time
import datetime
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
#import all the functions we wrote ourselves
import import_ipynb
import Moritz Functions as functions

SyntaxError: invalid syntax (<ipython-input-1-3e71c3d980bc>, line 49)

## Importing and Preprocessing Data

Get the 3 provided csv and merge them into one Pandas-Dataframe.

In [ ]:
#Add Keyfigures 
arr1=pd.read_csv("Keyfigures.csv")
#Add Ratings
arr2=pd.read_csv("Ratings.csv")
#Add S&P500 Company List
arr3=pd.read_csv("SP500_CompanyList.csv")
#Combine Keyfigures and S&P500 Company List
rest=functions.combine_keyfigures_and_Companies(arr1,arr3)
#Add Ratings to the mapping
mapping=functions.combine_Ratings_and_Rest(rest,arr2)

Save the joint Dataframe for further use in a csv:

In [ ]:
#mapping.to_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\Clean Mapping.csv')

In [ ]:
#df = pd.read_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\Clean Mapping.csv', sep=';',
                #parse_dates=['adate', 'qdate', 'public_date', 'datadate'])
df = mapping
df=df.rename(columns={'public_date_x':'public_date'})
df=df.rename(columns={'TICKER_x':'TICKER'})
df=df.rename(columns={'TICKER_y':'tic'})
df= df.drop(['public_date_y'],1)

Deleting all the rows where splticrm has NA values.

In [ ]:
df = df[df['splticrm'].notna()]

In [ ]:
df = df[df['splticrm'] != 'D']
df = df[df['splticrm'] != 'CCC']
print(df.head())

In [ ]:
Y = pd.factorize(df["splticrm"])[0]
print(np.bincount(Y))
print(pd.factorize(df["splticrm"])[1])
Y = pd.DataFrame(Y, columns=["Rating as Factor"])
print(Y)

After taking a first look at the data we see that for ratings D and CCC we only have 4, respectively 2 observations. Therefore we delete these values due to the very low number of observations.

### Handling missing data
We decided to drop columns which contain more than 10'000 (corresponds to roughly 30%)  NA values or zero values.

In [3]:
NAs = df.isnull().sum() > 10000
Zeros =  (df == 0).sum() > 10000
delNAs = df.columns[NAs] #drops PEG_trailing
delZeros= df.columns[Zeros] #drops rd_sale, adv_sale, staff_sale
df = df.drop(delNAs, axis=1)
df = df.drop(delZeros, axis=1)
df.to_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Collection\ML_Finance-master\Clean Mapping.csv')

NameError: name 'df' is not defined

Create a column of Ratings which are lagging by 1 Month and merge it with the dataframe

In [ ]:
nest_list=functions.create_lagging_Ratings_1M(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Collection\ML_Finance-master\Clean Mapping.csv')
dframe=pd.DataFrame(nest_list,columns=['key','Lagging-Rating_1M'])
#Replace the Nonetype from python with NaN from pandas
dframe.fillna(value=pd.np.nan, inplace=True)
#Now merge the new Dataframe to our main-df
df = df.merge(dframe, how='left', on=['key'])
df.head()

Assign to X all the columns but splticrm. Then we drop some columns which will not be relevant for the algorithms.

In [ ]:
X = df.loc[:, df.columns != "splticrm"]
X = X.drop(["permno", "CUSIP", "NCUSIP", "adate", "qdate", "public_date", "TICKER"], axis=1)
X = X.drop(["COMNAM", "PERMCO", "gvkey", "datadate", "tic", "cusip", "conm", "PRC"], axis=1)
X.head()

### Handling missing data
We decided to drop columns which contain more than 10'000 (corresponds to roughly 30%)  NA values or zero values.

Now we split the data into a train and a test set. The test set consists of 20% of the whole dataset.

In [2]:
#now do the train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=0, 
                                                    stratify=Y)

NameError: name 'train_test_split' is not defined

In [ ]:
X_train.head(1)


In [ ]:
X_train.isna().sum()

We see that there are still missing values that have to be imputed. We do this in the next section.

#### Iterative Imputer
The iterative imputer imputes missing values by by modeling each feature containing missing values as a function of other features and is applied separately to the train and test set.

In [ ]:
# apply IterativeImputer
## this can be deleted once the functions file is imported

num_cols = ['CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 
            'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
            'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
            'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt', 'int_totdebt', 'cash_lt', 'invt_act',
            'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct', 'ocf_lct',
            'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets', 'debt_capital', 'de_ratio', 'intcov',
            'intcov_ratio', 'cash_ratio', 'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
            'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc', 'accrual', 'ptb',
            'DIVYIELD', 'PEG_1yrforward', 'PEG_ltgforward']

# Copy df to df_mice_imputed
X_train_imputed = X_train[num_cols].copy(deep=True)

#Remove "%" from the DIVYIELD-Col
X_train_imputed['DIVYIELD'] = X_train_imputed['DIVYIELD'].str.rstrip('%').astype('float') / 100.0
#X_train_imputed.to_csv("Zwischencheck.csv")

# Initialize IterativeImputer
#mice_imputer = IterativeImputer(random_state=0)

# Impute using fit_tranform on diabetes
#X_train_imputed.iloc[:, :] = mice_imputer.fit_transform(X_train_imputed[num_cols])

# Copy df to df_mice_imputed
X_test_imputed = X_test[num_cols].copy(deep=True)
X_test_imputed['DIVYIELD'] = X_test_imputed['DIVYIELD'].str.rstrip('%').astype('float') / 100.0

# Impute using tranformation of training set on test set
#X_test_imputed.iloc[:, :] = mice_imputer.transform(X_test_imputed[num_cols])

In [ ]:
# apply IterativeImputer

X_train_imputed = functions.my_iterative_imputer(X_train_imputed)
X_test_imputed = functions.my_iterative_imputer(X_test_imputed)
X_train_imputed.head(5)

#### Create train and test data with lagging ratings

In [ ]:
#Save lagging ratings
lag_rating_train = X_train["Lagging-Rating_1M"]
lag_rating_test = X_test["Lagging-Rating_1M"]

#Merge imputed data with lagging ratings
X_train_imputed_lag = X_train_imputed.copy()
X_test_imputed_lag = X_test_imputed.copy()

X_train_imputed_lag["Lagging-Rating_1M"] = lag_rating_train
X_test_imputed_lag["Lagging-Rating_1M"] = lag_rating_test

#Define indetificator for train/test set in order to combine, modify and separate them again
X_train_imputed_lag["train"] = 1
X_test_imputed_lag["train"] = 0

#Combine both to factorize lagged ratings together
combined = pd.concat([X_train_imputed_lag, X_test_imputed_lag])

#Drop rows with NA in lagged rating
combined = combined[combined["Lagging-Rating_1M"].notna()]
combined["Lagging-Rating_1M"] = pd.factorize(combined["Lagging-Rating_1M"])[0]

#Separate train and test set
X_train_imputed_lag = combined[combined["train"] == 1]
X_test_imputed_lag = combined[combined["train"] == 0]
X_train_imputed_lag.drop(["train"], axis=1, inplace=True)
X_test_imputed_lag.drop(["train"], axis=1, inplace=True)

#Get the lagging ratings Dataframes Y for train and test set
y_train_lag = X_train_imputed_lag["Lagging-Rating_1M"]
y_test_lag = X_test_imputed_lag["Lagging-Rating_1M"]

y_train_lag = pd.DataFrame(y_train_lag)
y_test_lag = pd.DataFrame(y_test_lag)

#Drop Lagging Raating out of imputed feature matrix
X_train_imputed_lag.drop(["Lagging-Rating_1M"], axis=1, inplace=True)
X_test_imputed_lag.drop(["Lagging-Rating_1M"], axis=1, inplace=True)

y_train_lag.head(5)

### Feature Selection
#### For ordinary ratings
To find out which variables are most important we run the 'features_selection' function and select all variables which explain more than 1.5%.

In [ ]:
important_features = functions.feature_selection(x = X_train_imputed, y = y_train.values.ravel(), thres=0.015)

In [ ]:
#Subset important features matrix for ML algorithms
X_train_imputed = X_train_imputed.loc[:,important_features]
X_test_imputed = X_test_imputed.loc[:, important_features]
X_train_imputed.head(5)

In [ ]:
X_train_imputed.to_csv('X_train.csv')
X_test_imputed.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

#### For lagging ratings
Same approach as before. We select all the variables which explain more than 1.5%.

In [ ]:
important_features = functions.feature_selection(x = X_train_imputed_lag, y = y_train_lag.values.ravel(), thres=0.015)

In [ ]:
#Subset important features matrix for ML algorithms
X_train_imputed_lag = X_train_imputed_lag.loc[:,important_features]
X_test_imputed_lag = X_test_imputed_lag.loc[:, important_features]
X_train_imputed_lag.head(5)

In [ ]:
X_train_imputed_lag.to_csv('X_train_lag.csv')
X_test_imputed_lag.to_csv('X_test_lag.csv')
y_train_lag.to_csv('y_train_lag.csv')
y_test_lag.to_csv('y_test_lag.csv')

# Machine Learning Algorithms
## Ordinary Ratings

### Logistic Regression

In [ ]:
## Logistic Regression

X_train = pd.read_csv("X_train.csv")
X_train = X_train.iloc[:, 1:]
X_test = pd.read_csv("X_test.csv")
X_test = X_test.iloc[:, 1:]
Y_train = pd.read_csv("y_train.csv")
Y_test = pd.read_csv("y_test.csv")
Y_train = Y_train['Rating as Factor'].astype('category') #factorize trainset
Y_test = Y_test['Rating as Factor'].astype('category')   #factorize testset

In the grid of logreg__C:[6,6.5,7,7.5,8] and logreg__l1_ratio:[0,0.05,0.1,0.15,0.2,1] the best parameters were: {'logreg__C': 7, 'logreg__l1_ratio': 0, 'logreg__penalty': 'elasticnet'}

A ratio of 0 means that we are using the l2 penalty function (ridge regression regularization and not lasso)

In [ ]:
import datetime


param_grid = {'logreg__penalty': ['elasticnet'], #elastic nets combines l1&l2
              'logreg__C':[7],
              'logreg__l1_ratio':[0]} #if 0, or 1 then l2 or l1 would be best. If between then the combination of both

print(datetime.datetime.now())
grid = functions.LogReg(X_train,Y_train,param_grid)
print('Best CV accuracy: {:.4f}'.format(grid.best_score_)) #best parameters are C=7 & ratio=0 -> l2 penalty function
print('Test score:       {:.4f}'.format(grid.score(X_test, Y_test)))
print('Best parameters: {}'.format(grid.best_params_)) #31%
print(datetime.datetime.now()) #10 minutes

In [ ]:
# Predict classes
y_pred = grid.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))

### Support Vector Machines

##### Polynomial Kernel function: 
The gridsearch over {'svm_poly__C': [900,1000,1100], 'svm_poly__degree': [3,4,5], 'svm_poly__gamma': [0,0.05,0.1], 'svm_poly__coef0':[0.6]} 
resulted in {'svm_poly__C': 1000, 'svm_poly__coef0': 0.6, 'svm_poly__degree': 4, 'svm_poly__gamma': 0.05} being the best parameters. As the values are not at the gridsearch boundary we found a minima and can therefore stop the gridsearch. 

(Larger gridsearch beforehand yielded 0.6 to be the best coef0. As it does not greatly change the cv accuracy(<1%) we didn't include it in this grid search to lower the computing time.)

##### Radial Basis Kernel Function(rbf):
The gridsearch over {'svm_rbf__C': [100,150,200], 'svm_rbf__gamma': [0.25,0.3,0.35]} (and larger gridsearches beforehand) resulted in {'svm_rbf__C': 150, 'svm_rbf__gamma': 0.3} being the best parameters.

##### Radial Basis Kernel Function(rbf) with Balanced class weights:
The gridsearch over {'svm_rbf__C': [100,200,300], 'svm_rbf__gamma': [0.25,0.3,0.35]}  resulted in {'svm_rbf__C': 200, 'svm_rbf__gamma': 0.3} being the best parameters.

(According to "https://stackoverflow.com/questions/21390570/scikit-learn-svc-coef0-parameter-range" the Sigmoid function does not fulfill the definition of a kernel as it is not positive semidefinite. Therefore we will not use it with Support Vector Machines.)

In [ ]:
#######Polynomial Kernel Function#######
param_grid = {'svm_poly__C': [1000], 
              'svm_poly__degree': [4],
              'svm_poly__gamma': [0.05],
              'svm_poly__coef0':[0.6]}
    
print(datetime.datetime.now()) #computation time
poly = functions.SVM_poly(X_train,Y_train, param_grid)
print('Best CV accuracy: {:.4f}'.format(poly.best_score_))
print('Test score:       {:.4f}'.format(poly.score(X_test, Y_test)))
print('Best parameters: {}'.format(poly.best_params_))
print(datetime.datetime.now()) 

# Predict classes
y_pred = poly.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 


#######Radial Basis Kernel Function(rbf)#######
param_grid = {'svm_rbf__C': [150], 
              'svm_rbf__gamma': [0.3]} 
print(datetime.datetime.now()) #computation time
rbf = functions.SVM_rbf(X_train,Y_train,param_grid)
print('Best CV accuracy: {:.4f}'.format(rbf.best_score_))
print('Test score:       {:.4f}'.format(rbf.score(X_test, Y_test)))
print('Best parameters: {}'.format(rbf.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = rbf.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Radial Basis Function Kernel yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))


#######Radial Basis Kernel Function(rbf) with Balanced class weights#######
param_grid = {'svm_rbf__C': [200], 
              'svm_rbf__gamma': [0.3]} 
print(datetime.datetime.now()) #computation time
rbf_bal = functions.SVM_rbf_bal(X_train,Y_train,param_grid)
print('Best CV accuracy: {:.4f}'.format(rbf_bal.best_score_))
print('Test score:       {:.4f}'.format(rbf_bal.score(X_test, Y_test)))
print('Best parameters: {}'.format(rbf_bal.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = rbf_bal.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Radial Basis Function Kernel with Balanced class weights yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))


Looking at the confusion matrix of non-balanced rbf we see that the smaller classes don't get more wrong classification. Therefore balancing the weights should not influence the outcome greatly which it doesn't


### Random Forest

In [ ]:
#Define arrays of values to be tested for paramgrid inside GridSearchCV function

#number of trees in the forest
n_estimators = np.array([70])

# Max depth
maxDepth = np.array([25])

# Minimum number of samples required to split any internal node 
minSamplesNode = np.array([2])

# The minimum number of samples required to be at a leaf/terminal node
minSamplesLeaf = np.array([1])

In [ ]:
#Run RandomForest Classifier
import datetime
print(datetime.datetime.now())
forest = functions.random_forest(X_train,Y_train, n_estimators, maxDepth, minSamplesNode, minSamplesLeaf)
print('Best CV accuracy: {:.4f}'.format(forest.best_score_))
print('Test score:       {:.4f}'.format(forest.score(X_test,Y_test)))
print('Best parameters: {}'.format(forest.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = forest.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 

### MLP Classifier (Neural Network)

In [ ]:
#Define arrays of values to be tested for paramgrid inside GridSearchCV function

#The ith element represents the number of neurons in the ith hidden layer.
hidden_lay = np.array([400])

# Max number of iterations
max_Iter = np.array([575])

In [ ]:
#Run MLP Classifier
import datetime
print(datetime.datetime.now())
mlp = functions.neural(X_train,Y_train, hidden_lay, max_Iter)
print('Best CV accuracy: {:.4f}'.format(mlp.best_score_))
print('Test score:       {:.4f}'.format(mlp.score(X_test,Y_test)))
print('Best parameters: {}'.format(mlp.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = mlp.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 

### Neural Network

As pipelining a grid search doesn't work with the keras wrapper for sklearn we first have to manually standardize the data

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train) #standardizing using StandardScaler
X_test_std = scaler.transform(X_test) #standardizing using StandardScaler

The best Grid Search would be to test all hyperparameter configurations simultaneously because of possible interactions of the hyperparameters. 
However this would take an extremely long time due to the many hyperparameters and their possible values.
Therefore we take a sequential approach(1 or 2 hyperparameters at a time) on grid searching the hyperparameters to not have to train the model over multiple days.

##### Steps of grid search for hyperparameters:
After 100 epochs the accuracy doesn't improve much therefore we use epoch = 100 in order to have a well trained neural net which doesn't need too much time to fit.

batch_size = [5,10,20] yields that 5 is the best hyperparameter. It makes sense that a lower batch size results in a better accuracy since it optimizes more often. 
A lower batch size means we more often adjust the weights which takes time. 
The differences between the batch size accuracy is only 1% while the time for computing is significantly lower for higher batch sizes. 
Therefore for finding the other optimal hyperparameters we use a batch size of 20 and might adjust it for the final model fitting with the optimal hyperparameters

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'] yielded Nadam to be the best optimizer with Adam on second rank.
Nadam is is Adam with Nesterov Momentum as they both take the same amount of time to train we'll use Nadam because of its better accuracy.

Next up we split our gridsearch into a single layered and a two layered neural network.

##### One layer:
For the neural net with one layer we try to find the best combination of activation function and neurons as they interact heavily: Best combination of each activation function for neurons in grid[150,200,250,300]: 

Accuracy (Standard devation): 0.893909 (0.010722) with: {'activ': 'relu', 'neurons': 300}

Accuracy (Standard devation): 0.937349 (0.001372) with: {'activ': 'tanh', 'neurons': 250}

Accuracy (Standard devation): 0.933268 (0.000530) with: {'activ': 'sigmoid', 'neurons': 250

Accuracy (Standard devation): 0.897989 (0.006316) with: {'activ': 'hard_sigmoid', 'neurons': 300}

Accuracy (Standard devation): 0.767458 (0.032144) with: {'activ': 'softmax', 'neurons': 300}

Accuracy (Standard devation): 0.846942 (0.002463) with: {'activ': 'softplus', 'neurons': 300}

Accuracy (Standard devation): 0.935521 (0.003249) with: {'activ': 'softsign', 'neurons': 250}

Accuracy (Standard devation): 0.315170 (0.006817) with: {'activ': 'linear', 'neurons': 300}

We now test the Top 3 activation functions for 200 epochs with neurons grid[250,300,350,400] in hopes that maybe after 200 epochs one activation function will be clearly better:

Best is the Accuracy 0.947295 (0.000936) with: {'activ': 'sigmoid', 'neurons': 350} (1% better than other activation functions)

Next up we checked the dropoutrate combined with a weight restraint. The resulting best hyperparameters for the dropout rate were 0.0 and for the weight constraint the highest possible value. This means that regularization here is not needed and we shouldn't use dropouts or a weight constraint.
Checking dropoutrate together with weight constraint we get a dropout rate of 0.0 and a high weight constraint as best hyperparameters.
This means regularization here does not help and we don't use dropouts or weight constraints. We now have our final model.

##### Two layers:
For the two layered neural net we test the possible combinations of neurons with different activation functions in each layer .
We only included softsign, relu and sigmoid as the possible activation function such that the fitting of the net doesn't take days. We chose these 3 activation functions because sigmoid&softsign performed very well on the single layered neural net and relu seems to be favoured by a lot of people in the literature.
We didn't include tanh because it seems to work very similar to the sigmoid acivation function. 

Grid Search found this as the best combination: Accuracy: 0.943087 using {'activ1': 'sigmoid', 'activ2': 'sigmoid', 'neurons1': 400, 'neurons2': 150}

Again the grid search found that a dropout rate of 0.0 is the best combination in both layers. We now finished the two layered neural network.

In [ ]:
#1Layer Neural Net(According to the Universal Approximation Theorem theoretically a 1 Layer Neural Net can approximate any other Neural Network):
#Final model after Hyperparameter optimization. We get a 96% accuracy for the testset
dropout_rate = [0.0]
optimizer = ['Nadam']
activ = ['sigmoid']
neurons = [350]
epochs=[200]
batch_size=[10]
print(datetime.datetime.now())
# model fitting

param_grid = dict(dropout_rate=dropout_rate, optimizer=optimizer,activ=activ,neurons=neurons,epochs=epochs,batch_size=batch_size)
grid = functions.NeuralNet1layer(X_train_std,Y_train,param_grid)
#results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print('Test score:', grid.score(X_test_std,Y_test)) 
y_pred = grid.predict(X_test_std)
# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 
print(datetime.datetime.now())

In [ ]:
#Training the optimal Neural Network with 1 Layer on the whole Training Data
model = Sequential()
model.add(Dense(350, input_dim=36, activation='sigmoid'))
model.add(Dense(17, activation='softmax'))
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
# fit model
history = model.fit(X_train_std, Y_train, validation_data=(X_test_std, Y_test), epochs=200, batch_size=10, verbose=0)
# evaluate the model
_, train_acc = model.evaluate(X_train_std, Y_train, verbose=0)
_, test_acc = model.evaluate(X_test_std, Y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()


In [ ]:
#2Layer Neural Net:
activ1 = ['sigmoid']
activ2 = ['sigmoid']
neurons1 = [400]
neurons2 = [150]
dropout_rate1 = [0.0]
dropout_rate2 = [0.0]
print(datetime.datetime.now())
# model fitting

param_grid = dict(activ1=activ1,activ2=activ2,neurons1=neurons1,neurons2=neurons2,dropout_rate1=dropout_rate1,dropout_rate2=dropout_rate2)
grid = functions.NeuralNet2layer(X_train_std,Y_train,param_grid)
#results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print('Test score:', grid.score(X_test_std,Y_test)) 
y_pred = grid.predict(X_test_std)
# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 
print(datetime.datetime.now())

In [ ]:
#Training the optimal Neural Network with 2 Layers on the whole Training Data
model = Sequential()
model.add(Dense(400, input_dim=36, activation='sigmoid'))
model.add(Dense(150, activation='sigmoid'))
model.add(Dense(17, activation='softmax'))
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
# fit model
history = model.fit(X_train_std, Y_train, validation_data=(X_test_std, Y_test), epochs=200, batch_size=20, verbose=0)
# evaluate the model
_, train_acc = model.evaluate(X_train_std, Y_train, verbose=0)
_, test_acc = model.evaluate(X_test_std, Y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

As there was no improvement in accuracy of the two layered neural net compared to the single layered one we don't try to fit a three layered model. Also according to the universal approximation theorem a neural network with a single hidden layer containing a finite number of neurons can approximate any continuous functions and therefore a single layered network should be enough to fit our data.

## Lagging Ratings
### Logistic Regression

In [ ]:
# Logistic Regression

X_train_lag = pd.read_csv("X_train_lag.csv")
X_train_lag = X_train_lag.iloc[:, 1:]
X_test_lag = pd.read_csv("X_test_lag.csv")
X_test_lag = X_test_lag.iloc[:, 1:]
Y_train_lag = pd.read_csv("y_train_lag.csv")
Y_test_lag = pd.read_csv("y_test_lag.csv")
Y_train_lag = Y_train_lag['Lagging-Rating_1M'].astype('category') #factorize trainset
Y_test_lag = Y_test_lag['Lagging-Rating_1M'].astype('category')   #factorize testset

In [ ]:
import datetime
param_grid = {'logreg__penalty': ['elasticnet'], #elastic nets combines l1&l2
              'logreg__C':[7],
              'logreg__l1_ratio':[0]} #if 0, or 1 then l2 or l1 would be best. If between then the combination of both

print(datetime.datetime.now())
grid = functions.LogReg(X_train_lag,Y_train_lag,param_grid)
print('Best CV accuracy: {:.4f}'.format(grid.best_score_))
print('Test score:       {:.4f}'.format(grid.score(X_test_lag, Y_test_lag)))
print('Best parameters: {}'.format(grid.best_params_))
print(datetime.datetime.now())

In [ ]:
# Predict classes
y_pred = grid.predict(X_test_lag)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))

### Support Vector Machines

In [ ]:
#######Polynomial Kernel Function#######
param_grid = {'svm_poly__C': [1000], 
              'svm_poly__degree': [4],
              'svm_poly__gamma': [0.05],
              'svm_poly__coef0':[0.6]}
    
print(datetime.datetime.now()) #computation time
poly = functions.SVM_poly(X_train_lag,Y_train_lag,param_grid)
print('Best CV accuracy: {:.4f}'.format(poly.best_score_))
print('Test score:       {:.4f}'.format(poly.score(X_test_lag, Y_test_lag)))
print('Best parameters: {}'.format(poly.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = poly.predict(X_test_lag)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 


#######Radial Basis Kernel Function(rbf)#######
param_grid = {'svm_rbf__C': [150], 
              'svm_rbf__gamma': [0.3]} 
print(datetime.datetime.now()) #computation time
rbf = functions.SVM_rbf(X_train_lag,Y_train_lag,param_grid)
print('Best CV accuracy: {:.4f}'.format(rbf.best_score_))
print('Test score:       {:.4f}'.format(rbf.score(X_test_lag, Y_test_lag)))
print('Best parameters: {}'.format(rbf.best_params_))
print(datetime.datetime.now()) 

# Predict classes
y_pred = rbf.predict(X_test_lag)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print('Radial Basis Function Kernel yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))


#######Radial Basis Kernel Function(rbf) with Balanced class weights#######
param_grid = {'svm_rbf__C': [200], 
              'svm_rbf__gamma': [0.3]} 
print(datetime.datetime.now()) #computation time
rbf_bal = functions.SVM_rbf_bal(X_train_lag,Y_train_lag,param_grid)
print('Best CV accuracy: {:.4f}'.format(rbf_bal.best_score_))
print('Test score:       {:.4f}'.format(rbf_bal.score(X_test_lag, Y_test_lag)))
print('Best parameters: {}'.format(rbf_bal.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = rbf_bal.predict(X_test_lag)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print('Radial Basis Function Kernel with Balanced class weights yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))


### Random Forest

In [ ]:
#Define arrays of values to be tested for paramgrid inside GridSearchCV function

#number of trees in the forest
n_estimators = np.array([70])

# Max depth
maxDepth = np.array([25])

# Minimum number of samples required to split any internal node 
minSamplesNode = np.array([2])

# The minimum number of samples required to be at a leaf/terminal node
minSamplesLeaf = np.array([1])

In [ ]:
#Run RandomForest Classifier
import datetime
print(datetime.datetime.now())
forest = functions.random_forest(X_train_lag,Y_train_lag, n_estimators, maxDepth, minSamplesNode, minSamplesLeaf)
print('Best CV accuracy: {:.4f}'.format(forest.best_score_))
print('Test score:       {:.4f}'.format(forest.score(X_test_lag,Y_test_lag)))
print('Best parameters: {}'.format(forest.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = forest.predict(X_test_lag)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 

### MLP Classifier (Neural Network)

In [ ]:
#Define arrays of values to be tested for paramgrid inside GridSearchCV function

#The ith element represents the number of neurons in the ith hidden layer.
hidden_lay = np.array([400])

# Max number of iterations
max_Iter = np.array([575])

In [ ]:
#Run MLP Classifier
import datetime
print(datetime.datetime.now())
mlp = functions.neural(X_train_lag,Y_train_lag, hidden_lay, max_Iter)
print('Best CV accuracy: {:.4f}'.format(mlp.best_score_))
print('Test score:       {:.4f}'.format(mlp.score(X_test_lag,Y_test_lag)))
print('Best parameters: {}'.format(mlp.best_params_))
print(datetime.datetime.now())

# Predict classes
y_pred = mlp.predict(X_test_lag)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 

### Neural Network

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train_lag)
X_train_lag_std = scaler.transform(X_train_lag) #standardizing using StandardScaler
X_test_lag_std = scaler.transform(X_test_lag) #standardizing using StandardScaler

In [ ]:
#1Layer Neural Net(According to the Universal Approximation Theorem theoretically a 1 Layer Neural Net can approximate any other Neural Network):
#Final model after Hyperparameter optimization. We get a 96% accuracy for the testset
dropout_rate = [0.0]
optimizer = ['Nadam']
activ = ['sigmoid']
neurons = [350]
epochs=[200]
batch_size=[10]
print(datetime.datetime.now())
# model fitting

param_grid = dict(dropout_rate=dropout_rate, optimizer=optimizer,activ=activ,neurons=neurons,epochs=epochs,batch_size=batch_size)
grid = functions.NeuralNet1layer(X_train_lag_std,Y_train_lag,param_grid)
#results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print('Test score:', grid.score(X_test_lag_std,Y_test_lag)) 
y_pred = grid.predict(X_test_lag_std)
# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 
print(datetime.datetime.now())

In [ ]:
#Training the optimal Neural Network with 1 Layer on the whole Training Data
model = Sequential()
model.add(Dense(350, input_dim=36, activation='sigmoid'))
model.add(Dense(17, activation='softmax'))
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
# fit model
history = model.fit(X_train_lag_std, Y_train_lag, validation_data=(X_test_lag_std, Y_test_lag), epochs=200, batch_size=10, verbose=0)
# evaluate the model
_, train_acc = model.evaluate(X_train_lag_std, Y_train_lag, verbose=0)
_, test_acc = model.evaluate(X_test_lag_std, Y_test_lag, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()


In [ ]:
#2Layer Neural Net:
activ1 = ['sigmoid']
activ2 = ['sigmoid']
neurons1 = [400]
neurons2 = [150]
dropout_rate1 = [0.0]
dropout_rate2 = [0.0]
print(datetime.datetime.now())
# model fitting

param_grid = dict(activ1=activ1,activ2=activ2,neurons1=neurons1,neurons2=neurons2,dropout_rate1=dropout_rate1,dropout_rate2=dropout_rate2)
grid = functions.NeuralNet2layer(X_train_lag_std,Y_train_lag,param_grid)
#results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print('Test score:', grid.score(X_test_lag_std,Y_test_lag)) 
y_pred = grid.predict(X_test_lag_std)
# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test_lag})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 
print(datetime.datetime.now())

In [ ]:
#Training the optimal Neural Network with 2 Layers on the whole Training Data
model = Sequential()
model.add(Dense(400, input_dim=36, activation='sigmoid'))
model.add(Dense(150, activation='sigmoid'))
model.add(Dense(17, activation='softmax'))
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
# fit model
history = model.fit(X_train_std, Y_train, validation_data=(X_test_lag_std, Y_test_lag), epochs=200, batch_size=20, verbose=0)
# evaluate the model
_, train_acc = model.evaluate(X_train_lag_std, Y_train_lag, verbose=0)
_, test_acc = model.evaluate(X_test_lag_std, Y_test_lag, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

### Performance Metrics

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [ ]:
confm = confusion_matrix(y_test, y_pred)

In [ ]:
import itertools
plt.style.use('default')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#plt.style.use('default')
# Compute confusion matrix
confm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(confm, classes=['A', 'A+', 'AA-', 'AAA', 'BBB', 'BBB-', 'AA', 'A-', 'BBB+', 'AA+', 'CCC+', 
                                     'B-', 'B+', 'BB-', 'BB+', 'BB', 'B'],
                      title='Confusion matrix, without normalization');

In [ ]:
print(np.bincount(y_pred))
print(pd.factorize(df["splticrm"])[1])

In [ ]:
print('Accuracy:', metrics.accuracy_score(Y_test, y_pred))
print('Error Rate:', 1-metrics.accuracy_score(Y_test, y_pred))
print('Confusion Matrix:')
confm = pd.DataFrame({'Predicted': y_pred,'True': Y_test})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))

Print the precision, recall, f1-score, accuracy for each class. Support stands for number of samples for each class in Y_test. Macro avg is unweighted average between all rating groups. Weighted avg is weighted average between all rating groups. (weighted by support)

In [ ]:
print(metrics.classification_report(Y_test, y_pred, digits=3))

Despite not being very visually appealing, the following package shows all possible Performance Metrics one might want to use in the multiclass classification case.

In [ ]:
from pycm import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
Y_test_cm = np.array(Y_test)
cm = ConfusionMatrix(actual_vector=Y_test_cm, predict_vector=y_pred) # Create CM From Data
print(cm)

In [ ]:
FP = confm.sum(axis=0) - np.diag(confm)  
FN = confm.sum(axis=1) - np.diag(confm)
TP = np.diag(confm)
TN = confm.sum() - (FP + FN + TP)

# Metrics
Score = (TP+TN)/(TP+FP+FN+TN)
Errorrate = (FP+FN)/(TP+FP+FN+TN)
Specifity = TN/(TN+FP)
Sensivity = TP/(TP+FN)
Precision = TP/(TP+FP)
F1 = ((1+1**2)*TP)/((1+1**2)*TP+FN+FP)